In [42]:
import pandas as pd

# reading csv file 
df = pd.read_csv("Sample_Req.csv")

In [1]:
import os

os.environ["GROQ_API_KEY"] = "YOUR_API_KEY"

In [67]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
)

In [69]:
from langchain.prompts import PromptTemplate

template = '''
You are given a pandas data frame. You need to convert the data frame into JSON object.
1. Return a valid JSON. No preamble. 
2. The JSON object's keys should correspond to the column headers of the data frame, and each row should become a JSON object with its respective column values

Here is the data frame on which you need to perform this task:  
{data_frame}
'''

pt = PromptTemplate.from_template(template)

In [47]:
from langchain_core.output_parsers import JsonOutputParser

chain = pt | llm | JsonOutputParser()
json_output = chain.invoke(input={"data_frame": df})

In [53]:
json_output[1]

{'id': 208402, 'Primary Text': "iPDM ECU's behavior shall be defined based on ...", 'Artifact Type': 'SYS Requirement', 'Safety': 'ASIL B (D)', 'Legal': False, 'Domains': 'SYS,HW,SW', 'Status': 'accepted', 'Review State': 'pending', 'Verification_Criteria': 'Verify the NORMAL, LOW, OVER voltage scenarios...', 'Verification_Level': None, 'SYS_Verification_Level': 'SYS qual. test', 'isHeading': False}

In [54]:
from langchain_core.documents import Document

documents = [(Document(page_content=str(item), metadata={"Requirement ID": item["id"]})) for item in json_output]

In [8]:
# doc_dict = [{"page_content": doc.page_content, "metadata": doc.metadata} for doc in documents]

In [9]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [15]:
import uuid
from langchain_chroma import Chroma

ids = [str(uuid.uuid4()) for _ in range(len(documents))] 

unique_collection_name = f"collection_{uuid.uuid4()}"

client = Chroma(collection_name=unique_collection_name, embedding_function=embeddings_model)

# Add documents
client.add_documents(documents=documents, ids = ids)

['3ae37666-96ed-4ff5-ae9a-14e02af612dd',
 '9cc726dc-a3a5-48d0-b53e-0c9f28c466e1',
 '53404e28-acd3-4ce6-9bd3-5e48a9af325b',
 '3bdb4afd-7681-4a2a-b400-fdc5b3ae699f',
 'bf7a8a63-95a7-4a71-8fa6-193bb7232b27',
 'a61e07e2-142e-4717-a50d-a326810007f8',
 'ba8e2c1e-cc66-44c7-938b-9d1ea82dcd91',
 '273d44b8-8357-46d6-871e-808c7e81ecf7',
 '67e13311-c2b1-4d8d-9a27-cc1b699586c4',
 '5edbf8d6-9a3e-4fe8-a383-1519a10e361b',
 '9ac1beee-97ef-4e5e-842e-50af230f7272',
 '798fb590-e97d-44be-9d8a-58726988202e',
 'f2b6b523-1167-4e9e-84c8-031c33cdd839']

In [16]:
client.get()

{'ids': ['3ae37666-96ed-4ff5-ae9a-14e02af612dd',
  '9cc726dc-a3a5-48d0-b53e-0c9f28c466e1',
  '53404e28-acd3-4ce6-9bd3-5e48a9af325b',
  '3bdb4afd-7681-4a2a-b400-fdc5b3ae699f',
  'bf7a8a63-95a7-4a71-8fa6-193bb7232b27',
  'a61e07e2-142e-4717-a50d-a326810007f8',
  'ba8e2c1e-cc66-44c7-938b-9d1ea82dcd91',
  '273d44b8-8357-46d6-871e-808c7e81ecf7',
  '67e13311-c2b1-4d8d-9a27-cc1b699586c4',
  '5edbf8d6-9a3e-4fe8-a383-1519a10e361b',
  '9ac1beee-97ef-4e5e-842e-50af230f7272',
  '798fb590-e97d-44be-9d8a-58726988202e',
  'f2b6b523-1167-4e9e-84c8-031c33cdd839'],
 'embeddings': None,
 'documents': ["{'id': 206175, 'Primary Text': 'ECU Functioning Range', 'Artifact Type': 'Heading', 'Safety': None, 'Legal': None, 'Domains': None, 'Status': None, 'Review State': None, 'Verification_Criteria': None, 'Verification_Level': None, 'SYS_Verification_Level': None, 'isHeading': True}",
  '{\'id\': 208402, \'Primary Text\': "iPDM ECU\'s behavior shall be defined based on ...", \'Artifact Type\': \'SYS Requiremen

In [79]:
emb = client.get()["embeddings"]

In [23]:
all_documents = client.get()["documents"]

In [25]:
from langchain.prompts import PromptTemplate

template = ''' You are a test case generation assistant who will create test cases for system requirements of automotive applications. The requirements are stored in a vector store and provided to you in JSON object format.
Your task is to analyze a JSON object that contains various requirement details and generate relevant test cases based on certain conditions. Specifically, you should:
    Analyze the JSON Object:
        The input JSON structure contains keys such as 'Artifact Type', 'Verification_Criteria', 'Primary text' and so on.
        If only 'Artifact Type' is 'SYS Requirement', you should proceed with generating test cases based on the provided information or just skip  generating test case.

    Generate Test Cases:
        Review the 'Verification_Criteria' and the associated 'Primary text' to extract the necessary validation details.
        Create test cases that validate whether the system requirements match the 'Verification_Criteria' based on the description in 'Primary text'.
        The test cases should be structured, clear, and cover all possible validation scenarios as derived from the input.

    Output Requirements:
        The test case should be specific to the requirement described in the JSON object and must validate the system's behavior according to the 'Verification_Criteria'.
        Test cases should include clear Test Case ID(interger number followed by TC), Requirement covered("id" from input JSON object), objective, Pre-condition, steps for validation, expected results, and any necessary edge cases or exceptions.

If any part of the input is ambiguous, suggest potential clarifications.

Example JSON Input:

{{
  "id": 422911, "Primary Text": "If KL30_ALT > 16.5V, then iPDM shall set the DTC_Supply_voltage_exceeded flag.",
  "Artifact Type": "SYS Requirement",
  "Safety": "ASIL B (D)",
  "Legal": false,
  "Domains": "SYS",
  "Status": "accepted",
  "Review State": "pending",
  "Verification_Criteria": "Verify whether the DTC_Supply_voltage_exceeded flag is set when KL30_ALT exceeds 16.5V.",
  "Verification_Level": null,
  "SYS_Verification_Level": "SYS qual. test",
  "isHeading": false
}}

Expected Output:

    Test Case ID: TC1
    Requirement Covered: 422911
    Objective: Validate that the DTC_Supply_voltage_exceeded flag is set when KL30_ALT exceeds 16.5V.
    Pre-conditions:
    The system is powered on and operational.
    KL30_ALT voltage monitoring is active.
    Steps to Execute:
    1. Set the KL30_ALT voltage to 16V using a test tool.
    2. Read system fault memory.
    3. Set the KL30_ALT voltage to 16.6V using a test tool.
    4. Read system fault memory.
    Expected Results: 
    1. KL30_ALT voltage set to 16V.
    2. DTC_Supply_voltage_exceeded flag should not set.
    3. KL30_ALT voltage set to 16.6V
    4. DTC_Supply_voltage_exceeded flag should set. 

You should generate the valid test cases for all the individual documents in vector store.


### No Preamble ###
        
Here is the vector store on which you need to perform this task:  
{vector_store}
'''

pt_testcase = PromptTemplate.from_template(template=template, input_varible=["vector_store"])

In [55]:
chain = pt_testcase | llm
response = chain.invoke(input={"vector_store": json_output})

In [56]:
result = response.content

In [57]:
print(result.strip())

To generate test cases for the given vector store, we will iterate over each JSON object and check if the 'Artifact Type' is 'SYS Requirement'. If it is, we will generate test cases based on the 'Verification_Criteria' and 'Primary Text'.

Here are the test cases for each 'SYS Requirement' in the vector store:

### Test Case 1: 
Test Case ID: TC1
Requirement Covered: 208402
Objective: Validate the NORMAL, LOW, OVER voltage scenarios.
Pre-conditions:
* The system is powered on and operational.
* Voltage monitoring is active.
Steps to Execute:
1. Set the voltage to a normal value (e.g., 12V) using a test tool.
2. Read system fault memory.
3. Set the voltage to a low value (e.g., 10V) using a test tool.
4. Read system fault memory.
5. Set the voltage to an over value (e.g., 18V) using a test tool.
6. Read system fault memory.
Expected Results: 
1. System operates normally at 12V.
2. System operates in low voltage mode at 10V.
3. System operates in over voltage mode at 18V.

### Test Case 

In [35]:
# # Step 1: Split the string into lines
# lines = result.strip().split("\n")

# # Step 2: Split each line into a list of values
# rows = [line.split(" | ") for line in lines]

# # Step 3: Create a DataFrame
# header = rows[0]  # First row as header
# data_rows = rows[1:]  # Remaining rows as data
# df = pd.DataFrame(data_rows, columns=header)

#df

In [ ]:
# # Save to Excel (optional)
# save_path = r"C:\Users\LJO2KOR\Desktop\MY_Learning\LLM_Project\Projects\test cases\TestCase_1.xlsx"
# df.to_excel(save_path, index=False)

In [65]:
json_output[0]['Primary Text']

'ECU Functioning Range'

In [71]:
from langchain.prompts import PromptTemplate

template = '''You are an AI assistant specialized in generating test cases for system requirements of automotive applications. 
You have access to a tool called `TC Generation Tool`, which generates structured test cases based on the details of a system requirement. This tool does not include preamble phrases in the Objective and Steps to Execute sections, making the test cases concise and directly actionable.

### Tool Details:
- **Name:** `TC Generation Tool`
- **Functionality:** Generates a concise and structured test case for a given system requirement, provided the `Artifact Type` is "SYS Requirement."
- **Inputs:** A dictionary with the following keys:
  - `id`: The requirent ID
  - `Primary Text`: The main description of the requirement.
  - `Verification Criteria`: The conditions or steps to validate the requirement.
  - `Artifact Type`: The type of requirement (e.g., "SYS Requirement").
- **Output:** A structured test case including:
  - `Test Case ID`: A unique identifier for the test case.
  - `Requirement Covered`: ID of the requirement.
  - `Objective`: A validation statement directly tied to the primary text.
  - `Pre-conditions`: Necessary system setup before executing the test case.
  - `Steps to Execute`: A concise list of steps derived from the primary text and verification criteria.
  - `Expected Results`: Expected value for corresponding to each Execute step.

### Instructions:
1. When the user provides a requirement in JSON object format, extract the `Primary Text`, `Verification Criteria`, and `Artifact Type` from their input.
2. If the `Artifact Type` is "SYS Requirement," pass the data to the `write_test_case` tool to generate a test case.
3. If the `Artifact Type` is not "SYS Requirement," No need for generating test case.
4. Present the generated test case to the user in a clear and organized format.
5. If any required details are missing, ask the user to provide the missing information before generating the test case.

### Example Interaction:
**User Input:**
id: 411411
Primary Text: The system must maintain functionality under low voltage conditions. 
Verification Criteria: Simulate low voltage and ensure the system operates without errors. 
Artifact Type: SYS Requirement."

**AI Process:**
- Parse the requirement into the tool's input format.
- Call the `write_test_case` tool.
- Present the generated test case to the user.

**AI Response:**
"Here is the test case generated for your requirement:

Test Case ID: TC1 
Requirement Covered: 411411.  
Objective: Validate: The system must maintain functionality under low voltage conditions.  
Pre-conditions: The system is powered on and in a known operational state.  
Steps to Execute:  
1. The system must maintain functionality under low voltage conditions.  
2. Simulate low voltage and ensure system operates without errors.  
3. Observe and record the system's responses.  
Expected Results: 
1. System should be fully functional.
2. System should operate without errors in low voltage state.
3. System should not have any faults.


Here is the the requirement on which you need to perform this task:  
{requirement}

'''

prompt_tool = PromptTemplate.from_template(template=template, input_varible=["requirement"])

In [72]:
# defining tool to generate test case

from langchain_core.tools import tool
from langchain.agents import Tool

# Define the tool for writing test cases
tools = [
    Tool(
        name="TC Generation Tool",
        description="Generate structured test cases for input data",
        func=lambda input: writetestcasetool(
        input["args"]["primary_text"], input["args"]["verification_criteria"], input["args"]["id"]
        )
    )
]

def writetestcasetool(primary_text,verification_criteria,id_val):
    """Usefull to generate test cases for automotive system requirements"""
    test_case_id = "TC_" + str(hash(primary_text))[:6]  # Generate a unique test case ID
    test_case = {
        "Test Case ID": f"Should Incrementmental Number with suffix TC",
        "Requirement Coverd": id_val,
        "Test Case Title": f"Test for {primary_text[:30]}...",  # Shortened primary text as title
        "Objective": f"Validate that the system meets the requirement: {primary_text}",
        "Pre-conditions": "Define the system's initial state required for the test.",
        "Steps": [
            f"Step 1: {verification_criteria.split(',')[0].strip()}",
            "Step 2: Additional steps based on requirement."
        ],
        "Expected Results": (
            f"Each test step is performed correctly, and the outcomes match the requirement's intent: {primary_text}. "
            f"Confirm proper system behavior as per verification criteria: {verification_criteria}."
        )
    }
    return json.dumps(test_case, indent=2)

In [73]:
requirement_example = {
    "id": "425162",
    "Primary Text": "The system must maintain functionality under low voltage conditions.",
    "Verification Criteria": "Simulate low voltage and ensure system operates without errors.",
    "Artifact Type": "SYS Requirement"
}

chain = prompt_tool | llm
response = chain.invoke(input={"requirement": all_documents})

In [76]:
print(response.content.strip())

To generate test cases for the given system requirements, I will process each requirement individually. Since the `Artifact Type` is "SYS Requirement" for most of the given requirements, I will use the `TC Generation Tool` to generate test cases for those requirements.

Here are the test cases for each "SYS Requirement":

### Test Case for Requirement 208402
* **Test Case ID:** TC1
* **Requirement Covered:** 208402
* **Objective:** Validate iPDM ECU's behavior based on given conditions.
* **Pre-conditions:** The system is powered on and in a known operational state.
* **Steps to Execute:**
  1. Set up the system to simulate NORMAL voltage scenarios.
  2. Set up the system to simulate LOW voltage scenarios.
  3. Set up the system to simulate OVER voltage scenarios.
  4. Verify the iPDM ECU's behavior in each scenario.
* **Expected Results:**
  1. The iPDM ECU behaves as expected in NORMAL voltage scenarios.
  2. The iPDM ECU behaves as expected in LOW voltage scenarios.
  3. The iPDM EC